In [1]:
# Dependencies
import numpy as np
import pandas as pd
import os

In [2]:
df = pd.read_csv(os.path.join("..", "data", "beta_frame.csv"))
df.head()

,Unnamed: 0,ac45bi501,ac45bi5010,ac45bi5011,ac45bi5012,ac45bi5013,ac45bi5014,ac45bi5015,ac45bi5016,ac45bi5017,...,ac45bi5019,ac45bi502,ac45bi503,ac45bi504,ac45bi505,ac45bi506,ac45bi507,ac45bi508,ac45bi509,raster_frame
0,1,120.0,197.0,53.0,2075.0,357.0,11.0,71.0,1018.0,77.0,...,971.0,111.0,40.0,5637.0,289.0,15.0,274.0,58.0,197.0,0
1,2,119.0,196.0,52.0,2088.0,361.0,11.0,71.0,1029.0,77.0,...,982.0,111.0,40.0,5636.0,287.0,13.0,274.0,57.0,196.0,0
2,3,121.0,198.0,53.0,2055.0,355.0,11.0,71.0,1012.0,76.0,...,965.0,111.0,40.0,5658.0,289.0,15.0,274.0,58.0,198.0,0
3,4,121.0,198.0,53.0,2038.0,352.0,11.0,71.0,1003.0,75.0,...,957.0,111.0,40.0,5684.0,290.0,15.0,275.0,59.0,198.0,0
4,5,117.0,194.0,49.0,2082.0,362.0,11.0,72.0,1033.0,75.0,...,987.0,111.0,40.0,5682.0,287.0,11.0,276.0,54.0,194.0,0


In [3]:
X = df.drop("raster_frame", axis=1).fillna(0)
y = df["raster_frame"]
print(X.shape, y.shape)

(1688832, 20) (1688832,)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

C:\Users\bruce\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras.models import Sequential
from keras.layers import Dense

# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [6]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=10,
    shuffle=True,
    verbose=2
)

Epoch 1/10
 - 33s - loss: 0.1280 - acc: 0.9522
Epoch 2/10
 - 34s - loss: 0.1136 - acc: 0.9567
Epoch 3/10
 - 34s - loss: 0.1083 - acc: 0.9586
Epoch 4/10
 - 36s - loss: 0.1048 - acc: 0.9596
Epoch 5/10
 - 35s - loss: 0.1026 - acc: 0.9604
Epoch 6/10
 - 34s - loss: 0.1006 - acc: 0.9611
Epoch 7/10
 - 34s - loss: 0.0991 - acc: 0.9616
Epoch 8/10
 - 36s - loss: 0.0980 - acc: 0.9622
Epoch 9/10
 - 34s - loss: 0.0968 - acc: 0.9626
Epoch 10/10
 - 33s - loss: 0.0960 - acc: 0.9629


In [7]:
# Save the model
model.save(os.path.join("..", "data", "fire_trained.h5"))


In [8]:
# Load the model
from keras.models import load_model
voice_model = load_model(os.path.join("..", "data", "fire_trained.h5"))

In [9]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.09616189752585398, Accuracy: 0.9629329619524026


In [10]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

C:\Users\bruce\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [11]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: [0 0 0 0 0]
Actual Labels: [0, 0, 0, 0, 0]
